### 1. Import Libraries

In [ ]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import os

### 2. Data Obtention

In [ ]:
tickers = yf.Tickers('ACNT FRD MSB RS')
hist = tickers.history(start='2021-01-01',end='2024-12-31')
print('Data:')
print(hist)

[*********************100%***********************]  4 of 4 completed

Data:
Price       Close                                   Dividends                  \
Ticker       ACNT        FRD        MSB          RS      ACNT   FRD  MSB   RS   
Date                                                                            
2021-01-04   8.00   6.512208  16.668919  112.135773       0.0  0.00  0.0  0.0   
2021-01-05   8.01   6.676218  17.464701  115.348763       0.0  0.00  0.0  0.0   
2021-01-06   8.25   6.801639  17.495073  122.748901       0.0  0.00  0.0  0.0   
2021-01-07   8.73   6.927416  17.173115  124.060326       0.0  0.02  0.0  0.0   
2021-01-08   8.95   6.937090  16.772184  122.983070       0.0  0.00  0.0  0.0   
...           ...        ...        ...         ...       ...   ...  ...  ...   
2024-12-23  11.01  15.884322  22.506735  270.079987       0.0  0.00  0.0  0.0   
2024-12-24  11.01  15.804552  23.372063  270.980011       0.0  0.00  0.0  0.0   
2024-12-26  11.36  15.814523  23.569851  271.299988       0.0  0.00  0.0  0.0   
2024-12-27  11.28  15.

In [ ]:
adj_close = hist['Close']
print('Adj Close:')
print(adj_close)

Adj Close:
Ticker       ACNT        FRD        MSB          RS
Date                                               
2021-01-04   8.00   6.512208  16.668919  112.135773
2021-01-05   8.01   6.676218  17.464701  115.348763
2021-01-06   8.25   6.801639  17.495073  122.748901
2021-01-07   8.73   6.927416  17.173115  124.060326
2021-01-08   8.95   6.937090  16.772184  122.983070
...           ...        ...        ...         ...
2024-12-23  11.01  15.884322  22.506735  270.079987
2024-12-24  11.01  15.804552  23.372063  270.980011
2024-12-26  11.36  15.814523  23.569851  271.299988
2024-12-27  11.28  15.525354  22.877588  269.730011
2024-12-30  11.25  15.016817  23.174273  266.220001

[1004 rows x 4 columns]


In [ ]:
if os.path.exists('data.xlsx'):
    os.remove('data.xlsx')
adj_close.to_excel('data.xlsx')

### 3. Portfolio Analysis

In [ ]:
adj_close_values = adj_close.values
print('Adj Close values:')
print(np.round(adj_close_values,4))
print(adj_close_values.shape)

Adj Close values:
[[  8.       6.5122  16.6689 112.1358]
 [  8.01     6.6762  17.4647 115.3488]
 [  8.25     6.8016  17.4951 122.7489]
 ...
 [ 11.36    15.8145  23.5699 271.3   ]
 [ 11.28    15.5254  22.8776 269.73  ]
 [ 11.25    15.0168  23.1743 266.22  ]]
(1004, 4)


In [ ]:
R = np.log(adj_close_values[1:] / adj_close_values[:-1])
print('R:')
print(np.round(R*100, 4), '%')

R:
[[ 0.1249  2.4873  4.6636  2.825 ]
 [ 2.9522  1.8612  0.1738  6.2181]
 [ 5.6552  1.8323 -1.8574  1.0627]
 ...
 [ 3.1294  0.0631  0.8427  0.118 ]
 [-0.7067 -1.8454 -2.9811 -0.5804]
 [-0.2663 -3.3304  1.2885 -1.3098]] %


In [ ]:
RE = np.mean(R, axis=0)*252
RI = np.std(R, axis=0)*np.sqrt(252)
print('RE:', np.round(RE*100, 4), '%')
print('RI:', np.round(RI*100, 4), '%')

RE: [ 8.5657 20.9914  8.2785 21.7231] %
RI: [46.0564 55.4292 48.4815 28.9737] %


In [ ]:
Sharpes = RE / RI
print('Sharpes:', np.round(Sharpes, 4))

Sharpes: [0.186  0.3787 0.1708 0.7498]


In [ ]:
S = np.cov(R, rowvar=False)
print('S:')
print(S*100, '%')

S:
[[0.08425811 0.01775079 0.01326316 0.01164957]
 [0.01775079 0.1220422  0.01306287 0.01542646]
 [0.01326316 0.01306287 0.09336495 0.01302366]
 [0.01164957 0.01542646 0.01302366 0.03334587]] %


In [ ]:
corr = np.corrcoef(R, rowvar=False)
print('Correlation:')
print(corr)

Correlation:
[[1.         0.17504791 0.14953717 0.21977742]
 [0.17504791 1.         0.1223747  0.24181889]
 [0.14953717 0.1223747  1.         0.23341031]
 [0.21977742 0.24181889 0.23341031 1.        ]]


In [ ]:
n_assets = R.shape[1]
weights = np.ones(n_assets) / n_assets
print('Weights:', np.round(weights, 4))
print('Sum of weights:', np.sum(weights))

Weights: [0.25 0.25 0.25 0.25]
Sum of weights: 1.0


In [ ]:
def rep(w,r):
    return w @ np.transpose(r)

def varp(w,s):
    return w @ s @ np.transpose(w)

In [ ]:
ReP = rep(weights, RE)
varP = varp(weights, S)
RiP = np.sqrt(varP)*np.sqrt(252)
SharpeP = ReP / RiP
print('Portafolio pre-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio pre-optimización:
ReP: 14.8897 %
varP: 0.0003
RiP: 28.1007 %
SharpeP: 0.5299


### 4. Portfolio Optimization

In [ ]:
def constr(w):
    return np.sum(w) - 1

num_assets = 4
bounds = [(0, None)] * num_assets

constraints = [{'type': 'eq', 'fun': constr}]

res = minimize(
    fun=lambda w: varp(w, S),
    x0=weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True, 'maxiter': 1000, 'ftol': 1e-12}
)

print("Éxito:", res.success)
print("Mensaje:", res.message)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0002498381415269466
            Iterations: 19
            Function evaluations: 95
            Gradient evaluations: 19
Éxito: True
Mensaje: Optimization terminated successfully


In [ ]:
ReP = rep(res.x, RE)
varP = varp(res.x, S)
RiP = np.sqrt(varP)*np.sqrt(252)
SharpeP = ReP / RiP
print('Portafolio post-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio post-optimización:
ReP: 17.3895 %
varP: 0.0002
RiP: 25.0917 %
SharpeP: 0.693
